In [1]:
from load_results import load_result_dataset
import pandas as pd

pn1 = 'full_fine_tuning_50epochs_edge_paper_final2'
pn2 = 'full_fine_tuning_50epochs_paper_final2'
pn3 = 'none'
final_data1 = load_result_dataset(pn1, pn2, pn3)
final_data1 = [{**d, 'ft_strategy': 'FFT (50 epochs)'} for d in final_data1]
df1 = pd.DataFrame(final_data1)

pn1 = 'full_fine_tuning_5epochs_edge_article1'
pn2 = 'full_fine_tuning_5epochs_article1'
pn3 = 'none'
final_data2 = load_result_dataset(pn1, pn2, pn3)
final_data2 = [{**d, 'ft_strategy': 'FFT (5 epochs)'} for d in final_data2]
df2 = pd.DataFrame(final_data2)

pn1 = 'linearprobe_50epochs_edge_paper_final2'
pn2 = 'linearprobe_50epochs_paper_final2'
pn3 = 'none'
final_data3 = load_result_dataset(pn1, pn2, pn3)
final_data3 = [{**d, 'ft_strategy': 'LP (50 epochs)'} for d in final_data3]
df3 = pd.DataFrame(final_data3)

df = pd.concat([ df1, df2, df3 ], axis=0, ignore_index=True) 

final_data = []
final_data.extend(final_data1)
final_data.extend(final_data2)
final_data.extend(final_data3)

../results/none/CLIP-convnext_base_w-laion_aesthetic-s13B-b82K_uc-merced-land-use-dataset_TRADES_v2.pkl
HEY
../results/full_fine_tuning_50epochs_paper_final2/CLIP-convnext_base_w-laion_aesthetic-s13B-b82K_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/none/CLIP-convnext_base_w-laion2B-s13B-b82K_uc-merced-land-use-dataset_TRADES_v2.pkl
HEY
../results/full_fine_tuning_50epochs_paper_final2/CLIP-convnext_base_w-laion2B-s13B-b82K_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/none/deit_small_patch16_224.fb_in1k_uc-merced-land-use-dataset_TRADES_v2.pkl
HEY
../results/full_fine_tuning_50epochs_paper_final2/deit_small_patch16_224.fb_in1k_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/none/robust_resnet50_uc-merced-land-use-dataset_TRADES_v2.pkl
HEY
../results/full_fine_tuning_50epochs_paper_final2/robust_resnet50_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/none/vit_small_patch16_224.augreg_in21k_uc-merced-land-use-dataset_TRADES_v2.pkl
HEY
../results/full_fine_tuning_50

In [12]:

from process_database import process_grouped_df, process_rankings
import pandas as pd
import numpy as np
from pathlib import Path
pd.set_option("display.max_colwidth", None)   # no truncation for any column

# ── helper: make a two‑line cell ────────────────────────────────────────────
def fmt_entry(backbone: str, loss: str, rank: int, bs:int, ss:float) -> str:
    backbone = backbone.replace('_', r'\_')
    loss     = loss.replace('_', r'\_')
    return (
        r'\makecell{'
        f'{backbone}, {loss} \\\\[0.3ex] '     # ← four “\” in Python ⇒ “\\” in .tex
        r'\footnotesize (GR:' f'{rank}'  ', BS:' f'{bs}'  ', SS:' f'{ss}' r')'
        r'}'
    )

from pathlib import Path
import pandas as pd
import numpy as np

final_data = pd.concat([df1, df2, df3], ignore_index=True)

for setting in ("FFT (50 epochs)", "FFT (5 epochs)", "LP (50 epochs)" ): 
    rows = []
    d = final_data[final_data.ft_strategy == setting]
    g = process_rankings( process_grouped_df(d) )

    for size in ( "small","medium",  "large", ): # 
        copy = g.xs(size, level="model_size")

        # ── flatten & rename columns ────────────────────────────────────────
        cols_keep = ["rank_borda", "score_sum", "borda" ] #"rank_geom", "rank_sum", "score_geom",  "borda"
        df_bis = (copy.loc[:, ("TOTAL", cols_keep)].reset_index())

        df_bis.columns = ["_".join(c) if isinstance(c, tuple) else c for c in df_bis.columns]
        df_bis.columns = ["backbone", "backbone_name", "loss",
                      "pre_training_strategy", "model_type", "ft_strategy",
                      "rank_borda_tot",  "score_sum_tot", "score_borda_tot" ] #"rank_geom_tot", "rank_sum_tot", "score_geom_tot", 

        # ── choose top‑3 by total‑sum score ────────────────────────────────
        df_bis = (df_bis.round(4)
                .sort_values("rank_borda_tot", ascending=True)
                .head(3)
                .reset_index(drop=True))

        rows.append({
            "Size": size,
            "Gold (1st)"  : fmt_entry( df_bis.loc[0, "backbone_name"],
                                       df_bis.loc[0, "loss"],
                                       int(df_bis.loc[0, "rank_borda_tot"]),
                                       int(df_bis.loc[0, "score_borda_tot"]),
                                       round( float(df_bis.loc[0, "score_sum_tot"]), 2)   ),

            "Silver (2nd)": fmt_entry( df_bis.loc[1, "backbone_name"],
                                       df_bis.loc[1, "loss"],
                                       int(df_bis.loc[1, "rank_borda_tot"]),
                                       int(df_bis.loc[1, "score_borda_tot"]),
                                      round( float(df_bis.loc[1, "score_sum_tot"]), 2)   ),

            "Bronze (3rd)": fmt_entry( df_bis.loc[2, "backbone_name"],
                                       df_bis.loc[2, "loss"],
                                       int(df_bis.loc[2, "rank_borda_tot"]),
                                       int(df_bis.loc[2, "score_borda_tot"]),
                                       round( float(df_bis.loc[2, "score_sum_tot"]), 2)  ),
        })

    table = pd.DataFrame(rows).round(2)
    table.columns = [rf"\textbf{{{c}}}" for c in table.columns]

    body = table.to_latex(index=False,
                          escape=False,
                          column_format="|" + "|".join(["c"]*table.shape[1]) + "|")

    safe_label = (setting.replace(" ", "_")
                         .replace("(", "")
                         .replace(")", ""))

    # ── only wrap in \resizebox if the table is wide (> 1.0× textwidth) ──
    maybe_resize_open  = r"\resizebox{\textwidth}{!}{%"  # default on
    maybe_resize_close = r"}"

    # crude width check (char count) – feel free to refine
    approx_char_per_col = 18
    if approx_char_per_col * table.shape[1] < 140:   # fits? skip resizebox
        maybe_resize_open = maybe_resize_close = ""

    latex_table = "\n".join([
        r"\begin{table}[ht]",
        r"\centering",
        rf"\caption{{Top fine‑tuning configurations in {setting}, with global ranking (GR) based on Borda score (BS), Sum score (SS) also reported below. }}",
        rf"\label{{tab:{safe_label}}}",
        r"\resizebox{\textwidth}{!}{%",      #  ← open
        body,                                #  ← pandas output
        r"}",                                #  ← close
        r"\end{table}",
        ""
    ])

    out = Path("latex_tables") / f"{safe_label}.tex"
    out.write_text(latex_table, encoding="utf-8")
    print("Saved:", out)


Percentage of NaN values: 0.00%
Saved: latex_tables/FFT_50_epochs.tex
Percentage of NaN values: 0.00%
Saved: latex_tables/FFT_5_epochs.tex
Percentage of NaN values: 0.00%
Saved: latex_tables/LP_50_epochs.tex


/var/folders/v7/3s0lms795672_f7_mh2x6bcr0000gn/T/ipykernel_15232/2226545357.py:71: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  body = table.to_latex(index=False,
/var/folders/v7/3s0lms795672_f7_mh2x6bcr0000gn/T/ipykernel_15232/2226545357.py:71: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  body = table.to_latex(index=False,
/var/folders/v7/3s0lms795672_f7_mh2x6bcr0000gn/T/ipykernel_15232/2226545357.py:71: FutureWarning: In future versions `DataFrame.to_latex` is expected 